In [1]:
import great_expectations as gx
import pandas as pd

In [2]:
# Load data clean
df = pd.read_csv("P2M3_devano_fernando_data_clean.csv")

# Buat validator
context = gx.get_context()
validator = context.sources.pandas_default.read_dataframe(df)

print("Data loaded. Shape:", df.shape)

Data loaded. Shape: (9648, 13)


## 1. Unique Key

In [3]:
# Buat kolom unik: retailer + invoice_date + product
df["unique_key"] = (
    df["retailer_id"].astype(str) + "_" + 
    df["invoice_date"].astype(str) + "_" + 
    df["city"].astype(str) + "_" + 
    df["product"].astype(str) + "_" + 
    df["price_per_unit"].astype(str) + "_" + 
    df["units_sold"].astype(str)
)
validator = context.sources.pandas_default.read_dataframe(df)

validator.expect_column_values_to_be_unique("unique_key")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9648,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Hasil validasi dari Great Expectations menunjukkan tidak ada data transaksi yang duplikat di kolom unique_key. Hal ini dikarenakan kolom tersebut dibuat dari gabungan beberapa informasi seperti ID retailer, tanggal, kota, produk, harga, dan jumlah unit, yang mana memang seharusnya unik untuk setiap transaksi. Karena hasilnya success: true dan unexpected_count: 0, kita bisa menyimpulkan bahwa data yang dipakai untuk analisis di Kibana sudah akurat

## 2. To be between min_value and max_value (operating margin)

In [4]:
validator.expect_column_values_to_be_between(
    column="operating_margin",
    min_value=0,
    max_value=100
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9648,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Hasil dari validasi dapat kita simpulkan bahwa semua nilai operating margin dalam dataset sudah trus (akurat), yaitu dalam range 0-100%. Bisa diartikan juga bahwa nilai dari operating margin tidak ada yang minus atau diatas 100%. 

## 3. To be in set

In [5]:
validator.expect_column_values_to_be_in_set(
    column="sales_method",
    value_set=["Online", "In-store", "Outlet"]
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9648,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Hasil valdiasi dari kolom sales_method hanya berisi dari 3 kategori, yaitu hanya ada "Online", "In-store", dan "Outlet". Data nya juga bisa dikatakan bersih (tidak ada yang typo, data kosong, atau kategori selain ketiga tersebut)

## 4. to be in type list

In [6]:
validator.expect_column_values_to_be_of_type("units_sold", "float") 

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Hasil validasi menunjukkan bahwa kolom units_sold sudah berhasil diubah tipe datanya menjadi float. Format teks seperti "1200" sudahbisa di convert menjadi angka desimal (bukan lagi teks).

##  5. Tidak ada missing di kolom retailer (custom #1)

In [7]:
validator.expect_column_values_to_not_be_null("retailer")

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9648,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Dapat disimpulkan bahwa hasil dari validasi kolom retailer sudah tidak ada yang missing values. Semua data sudah terisi 

## 6. Jumlah baris tetap 9648 (custom #2)

In [8]:
validator.expect_table_row_count_to_equal(9648)

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 9648
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Hasil dari validasi menunjukan bahwa jumlah datanya tetap konsisten, dimana tetap berjumlah 9.684 setelah dilakukan cleaning. Bisa diartikan tidak ada data transaksi yang di drop

## 7. Rata-rata operating margin antara 25–55 (custom #3)

In [9]:
validator.expect_column_mean_to_be_between(
    column="operating_margin",
    min_value=25,
    max_value=55
)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 42.29912935323383
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Hasil validasi ini menunjukan bahwa average dari nilai operating margin di dataset ada di dalam range 25%-55%. Dengan average yang bernilai 42.3%, bisa dikatakan bahwa profitabilitas dalam bisnisnya tergolong sehat (tidak terlalu tinggi yang menandakan tidak wajar, ataupun tidak terllau rendah yang dalam artian bisa rugi). Bisa dikatakan juga bahwa strategi dari harga dan operating margin sudah cukup balance.